## CRUD con Embeddings

Instalar dependencias en caso de no haber ejecutado el archivo requirements.txt

In [1]:
%pip install sentence-transformers numpy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Importar librerías necesarias

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\Solvo\Documents\vero\CRUD python\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Crear el modelo de embeddings

In [3]:
modelo = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Solvo\Documents\vero\CRUD python\env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Solvo\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)



Inicializar el DataFrame donde se almacenarán los embeddings

In [4]:
embeddings_df = pd.DataFrame(columns=["id", "embedding", "texto"])

1. Crear un embedding y agregarlo al DataFrame

In [6]:
def agregar_embedding(texto):
    embedding = modelo.encode([texto])[0]
    nuevo_id = len(embeddings_df) + 1
    embeddings_df.loc[len(embeddings_df)] = [nuevo_id, embedding, texto]
    return nuevo_id, embedding

2. Leer un embedding (consultar todos los embeddings)

In [7]:
def obtener_embeddings():
    return embeddings_df

3. Consultar los embeddings más similares a una consulta

In [9]:
def consultar_similares(texto_consulta, top_k=5):
    embedding_consulta = modelo.encode([texto_consulta])[0]
    embeddings_array = np.array(embeddings_df['embedding'].tolist())
    sim = cosine_similarity([embedding_consulta], embeddings_array)
    indices_similares = sim[0].argsort()[-top_k:][::-1]
    
    resultados = []
    for idx in indices_similares:
        resultados.append(embeddings_df.iloc[idx])
    
    return resultados

4. Actualizar un embedding por ID

In [89]:
def actualizar_embedding(id, nuevo_texto):
    try:
        # Verificar el tipo de la columna 'embedding'
        if embeddings_df['embedding'].dtype != 'object':
            embeddings_df['embedding'] = embeddings_df['embedding'].astype(object)
        
        # Obtener el embedding antiguo
        embedding_antiguo = embeddings_df.loc[embeddings_df['id'] == id, 'embedding'].values[0]
        print("Tamaño del antiguo embedding:", len(embedding_antiguo))

        # Generar el nuevo embedding
        embedding_nuevo = modelo.encode([nuevo_texto])[0]
        print("Tamaño del nuevo embedding:", len(embedding_nuevo))

        # Validar tamaño y tipo del embedding
        if isinstance(embedding_nuevo, (list, np.ndarray)) and len(embedding_nuevo) == len(embedding_antiguo):
            # Actualizar la fila correspondiente en el DataFrame
            embeddings_df.loc[embeddings_df['id'] == id, 'embedding'] = embeddings_df.loc[embeddings_df['id'] == id, 'embedding'].apply(
                lambda _: embedding_nuevo.tolist() if isinstance(embedding_nuevo, np.ndarray) else embedding_nuevo
            )
            embeddings_df.loc[embeddings_df['id'] == id, 'texto'] = nuevo_texto
            print(f"Embedding actualizado correctamente para el ID {id}")
            return id, nuevo_texto
        else:
            raise ValueError("El tamaño del nuevo embedding no coincide con el tamaño esperado.")
    except Exception as e:
        print(f"Error al actualizar el embedding: {e}")
        return None, None



5. Eliminar un embedding por ID

In [11]:
def eliminar_embedding(id):
    embeddings_df.drop(embeddings_df[embeddings_df['id'] == id].index, inplace=True)
    return id

# Uso del CRUD

1. Crear embeddings

In [37]:
id1, embedding1 = agregar_embedding("Este es un texto de prueba.")
id2, embedding2 = agregar_embedding("Otro texto para generar un embedding.")
id3, embedding3 = agregar_embedding("Ejemplo de texto para el crud.")

2. Leer los embeddings

In [96]:
print("Embeddings en la base de datos:")
print(obtener_embeddings())

Embeddings en la base de datos:
Empty DataFrame
Columns: [id, embedding, texto]
Index: []


3. Consultar los embeddings más similares a un nuevo texto

In [44]:
resultados = consultar_similares("texto de ejemplo")
print("\nEmbeddings más similares a la consulta:")
for resultado in resultados:
    print(resultado)


Embeddings más similares a la consulta:
id                                                           3
embedding    [-0.008501961, 0.11116948, -0.060942918, 0.000...
texto                           Ejemplo de texto para el crud.
Name: 2, dtype: object
id                                                           1
embedding    [-0.017046897, 0.07555656, -0.038786806, 0.034...
texto                              Este es un texto de prueba.
Name: 0, dtype: object
id                                                           2
embedding    [-0.060038242, -0.006098334, -0.011147308, -0....
texto                    Otro texto para generar un embedding.
Name: 1, dtype: object


4. Actualizar un embedding

In [90]:
# ID del embedding a actualizar y nuevo texto
id_actualizar = 2
nuevo_texto = "Un texto generado para comprobar el funcionamiento del embedding."

# Llamar a la función para actualizar el embedding
id_actualizado, texto_actualizado = actualizar_embedding(id_actualizar, nuevo_texto)

if id_actualizado is not None:
    print(f"Embedding actualizado: {id_actualizado}, Nuevo texto: {texto_actualizado}")
else:
    print("Hubo un error al actualizar el embedding.")


Tamaño del antiguo embedding: 384
Tamaño del nuevo embedding: 384
Embedding actualizado correctamente para el ID 2
Embedding actualizado: 2, Nuevo texto: Un texto generado para comprobar el funcionamiento del embedding.


5. Eliminar un embedding

In [95]:
id = 3
id_eliminado = eliminar_embedding(id)
print(f"\nEmbedding eliminado: {id_eliminado}")


Embedding eliminado: 3
